In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 4.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=7dc9bde41284895b9902c329e8f33c1cebac59d0f6ce79d7be8ff8fd62aa356e
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=2614248b3ad8c8159eb1fcdcadc97ebe3ac035aa9e6f6b5ad9f4d32221cc654e
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [19]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [20]:
train_images=train_images/255.0
test_images=test_images/255.0

In [21]:
train_images[0].shape

(28, 28)

In [22]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [48]:
def build_model(hp):
  model = keras.Sequential([
             keras.layers.Conv2D(
                  filters=hp.Int('conv_1_filter',min_value=32, max_value=128, step=16),
                  kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
                  activation='relu',
                  input_shape=(28,28,1)
                  ),
                  keras.layers.Conv2D(
                      filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
                      kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
                      activation='relu'
                  ),
                  keras.layers.Flatten(),
                  keras.layers.Dense(
                     units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
                     activation='relu'
                  ),
                  keras.layers.Dense(10,activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

In [51]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [53]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name='Mnist Fashion')

In [57]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 6 Complete [00h 13m 22s]
val_accuracy: 0.8554999828338623

Best val_accuracy So Far: 0.909333348274231
Total elapsed time: 01h 44m 23s
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit


In [56]:
model=tuner_search.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        27696     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2230368   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,260,698
Trainable params: 2,260,698
Non-trainable params: 0
_________________________________________________________________
